In [1]:
import warnings
warnings.filterwarnings('ignore') # 忽略warning错误信息

from pathlib import Path
from time import time
from collections import deque
from random import sample

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [2]:
import gym
from gym.envs.registration import register

np.random.seed(42)
tf.random.set_seed(42)

# 将时间信息格式化
def format_time(t):
    m_, s = divmod(t, 60)
    h, m = divmod(m_, 60)
    return '{:02.0f}:{:02.0f}:{:02.0f}'.format(h, m, s)

# 生成训练数据目录
results_path = Path('results', 'trading_bot')
if not results_path.exists():
    results_path.mkdir(parents=True)


# 定义交易天数
trading_days = 252

# 注册环境 进口为 trading_env.py 文件中的TradingEnvironment类
register(
    id='trading-v0',
    entry_point='trading_env:TradingEnvironment', 
    max_episode_steps=trading_days
)

trading_environment = gym.make('trading-v0')
trading_environment.env.trading_days = trading_days
trading_environment.env.trading_cost_bps = 1e-3
trading_environment.env.time_cost_bps = 1e-4
trading_environment.env.ticker = 'AAPL'
trading_environment.seed(42)

# 环境参数
state_dim = trading_environment.observation_space.shape[0] 
num_actions = trading_environment.action_space.n
max_episode_steps = trading_environment.spec.max_episode_steps

INFO:trading_env:trading_env logger started.
INFO:trading_env:loading data for AAPL...
INFO:trading_env:got data for AAPL...
INFO:trading_env:None
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 9367 entries, (Timestamp('1981-01-30 00:00:00'), 'AAPL') to (Timestamp('2018-03-27 00:00:00'), 'AAPL')
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   returns  9367 non-null   float64
 1   ret_2    9367 non-null   float64
 2   ret_5    9367 non-null   float64
 3   ret_10   9367 non-null   float64
 4   ret_21   9367 non-null   float64
 5   rsi      9367 non-null   float64
 6   macd     9367 non-null   float64
 7   atr      9367 non-null   float64
 8   stoch    9367 non-null   float64
 9   ultosc   9367 non-null   float64
dtypes: float64(10)
memory usage: 1.7+ MB


In [4]:
print (state_dim) # 观测的维度
print (num_actions) # 动作数量
print (max_episode_steps) # 最大步数

10
3
252


In [3]:
# model path
# 生成训练数据目录
models_path = Path('models')

if not models_path.exists():
    models_path.mkdir(parents=True)
models_path

PosixPath('models')

In [4]:
# 定义交易agent
class DDQNAgent:
    def __init__(self, state_dim,
                 num_actions,
                 learning_rate,
                 gamma,
                 epsilon_start,
                 epsilon_end,
                 epsilon_decay_steps,
                 epsilon_exponential_decay,
                 replay_capacity,
                 architecture,
                 l2_reg,
                 tau,
                 batch_size):

        self.state_dim = state_dim                               # 10个数据
        self.num_actions = num_actions                           # 3个动作: long ,hold, short
        self.experience = deque([], maxlen=replay_capacity)      # 用于记录（s, a, s', r, done）
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.architecture = architecture
        self.l2_reg = l2_reg

        self.online_network = self.build_model()                 
        self.target_network = self.build_model(trainable=False)
        self.update_target()

        self.epsilon = epsilon_start
        self.epsilon_decay_steps = epsilon_decay_steps
        self.epsilon_decay = (epsilon_start - epsilon_end) / epsilon_decay_steps
        self.epsilon_exponential_decay = epsilon_exponential_decay
        self.epsilon_history = []

        self.total_steps = self.train_steps = 0
        self.episodes = self.episode_length = self.train_episodes = 0
        self.steps_per_episode = []
        self.episode_reward = 0
        self.rewards_history = []

        self.batch_size = batch_size
        self.tau = tau
        self.losses = []
        self.idx = tf.range(batch_size)
        self.train = True

    def build_model(self, trainable=True):
        layers = []
        n = len(self.architecture)
        for i, units in enumerate(self.architecture, 1):
            layers.append(Dense(units=units,
                                input_dim=self.state_dim if i == 1 else None,
                                activation='relu',
                                kernel_regularizer=l2(self.l2_reg),
                                name=f'Dense_{i}',
                                trainable=trainable))
        layers.append(Dropout(.1))
        layers.append(Dense(units=self.num_actions,
                            trainable=trainable,
                            name='Output'))
        model = Sequential(layers)
        model.compile(loss='mean_squared_error',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def update_target(self):
        self.target_network.set_weights(self.online_network.get_weights()) # 将online网络的权重赋值给target网络

    def epsilon_greedy_policy(self, state):
        self.total_steps += 1
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.num_actions)  # 以epsilon的概率随机的选择动作
        q = self.online_network.predict(state)         # 以1-epsilon的概率选择使 Q(s,a)最大的action
        return np.argmax(q, axis=1).squeeze()

    def memorize_transition(self, s, a, r, s_prime, not_done):
        if not_done:
            self.episode_reward += r
            self.episode_length += 1
        else:
            if self.train: # epsilon不断递减
                if self.episodes < self.epsilon_decay_steps:
                    self.epsilon -= self.epsilon_decay
                else:
                    self.epsilon *= self.epsilon_exponential_decay

            self.episodes += 1
            self.rewards_history.append(self.episode_reward)
            self.steps_per_episode.append(self.episode_length)
            self.episode_reward, self.episode_length = 0, 0

        self.experience.append((s, a, r, s_prime, not_done)) # 将(s, a, s_prime, not_done) 写入experience

    def experience_replay(self):
        if self.batch_size > len(self.experience):
            return
        minibatch = map(np.array, zip(*sample(self.experience, self.batch_size)))
        states, actions, rewards, next_states, not_done = minibatch

        next_q_values = self.online_network.predict_on_batch(next_states)
        best_actions = tf.argmax(next_q_values, axis=1)

        next_q_values_target = self.target_network.predict_on_batch(next_states)
        target_q_values = tf.gather_nd(next_q_values_target,
                                       tf.stack((self.idx, tf.cast(best_actions, tf.int32)), axis=1))

        targets = rewards + not_done * self.gamma * target_q_values

        q_values = self.online_network.predict_on_batch(states)
        q_values[[self.idx, actions]] = targets

        loss = self.online_network.train_on_batch(x=states, y=q_values)
        self.losses.append(loss)

        if self.total_steps % self.tau == 0:
            self.update_target()

    def save_model(self):
        try:
            self.online_network.save(models_path / 'online_model') 
            self.target_network.save(models_path / 'target_model')
            print('models have been saved.')
            return True
        except:
            return False

    def load_model(self):
        try:
            self.online_network = tf.keras.models.load_model(models_path / 'online_model') 
            self.target_network = tf.keras.models.load_model(models_path / 'target_model')
            print('models have been loaded.')
            return True
        except:
            return False

In [5]:
# 定义模型参数与网络结构
gamma = .99,  # discount factor
tau = 100  # target network update frequency

architecture = (256, 256)  # 神经网络的参数
learning_rate = 0.0001  # learning rate
l2_reg = 1e-6  # L2 regularization
replay_capacity = int(1e6)
batch_size = 4096

epsilon_start = 1.0
epsilon_end = .01
epsilon_decay_steps = 250
epsilon_exponential_decay = .99

tf.keras.backend.clear_session()

ddqn = DDQNAgent(state_dim=state_dim,
                 num_actions=num_actions,
                 learning_rate=learning_rate,
                 gamma=gamma,
                 epsilon_start=epsilon_start,
                 epsilon_end=epsilon_end,
                 epsilon_decay_steps=epsilon_decay_steps,
                 epsilon_exponential_decay=epsilon_exponential_decay,
                 replay_capacity=replay_capacity,
                 architecture=architecture,
                 l2_reg=l2_reg,
                 tau=tau,
                 batch_size=batch_size)

In [7]:
ddqn.online_network.summary()
ddqn.online_network.weights

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Dense_1 (Dense)              (None, 256)               2816      
_________________________________________________________________
Dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
Output (Dense)               (None, 3)                 771       
Total params: 69,379
Trainable params: 69,379
Non-trainable params: 0
_________________________________________________________________


[<tf.Variable 'Dense_1/kernel:0' shape=(10, 256) dtype=float32, numpy=
 array([[ 0.04943046, -0.01772013, -0.04419052, ...,  0.00709645,
          0.10010409,  0.08421537],
        [ 0.07523823,  0.08323127,  0.04633622, ..., -0.035027  ,
          0.09194687,  0.05923855],
        [-0.1310968 ,  0.07321371, -0.11101919, ..., -0.14396206,
         -0.0499901 , -0.09893271],
        ...,
        [ 0.14955643, -0.00583661, -0.00119951, ..., -0.07104123,
         -0.0286596 , -0.03342304],
        [ 0.06839566, -0.12877463,  0.05618842, ..., -0.01180017,
          0.09284012, -0.12714407],
        [-0.03903491, -0.09870157,  0.07117632, ..., -0.02694204,
         -0.08345815, -0.12888123]], dtype=float32)>,
 <tf.Variable 'Dense_1/bias:0' shape=(256,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [8]:
ddqn.save_model()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: models/online_model/assets
INFO:tensorflow:Assets written to: models/online_model/assets
INFO:tensorflow:Assets written to: models/target_model/assets
INFO:tensorflow:Assets written to: models/target_model/assets
models have been saved.


True

In [15]:
ddqn.load_model()
ddqn.online_network.weights

models have been loaded.


[<tf.Variable 'Dense_1/kernel:0' shape=(10, 256) dtype=float32, numpy=
 array([[ 0.04943046, -0.01772013, -0.04419052, ...,  0.00709645,
          0.10010409,  0.08421537],
        [ 0.07523823,  0.08323127,  0.04633622, ..., -0.035027  ,
          0.09194687,  0.05923855],
        [-0.1310968 ,  0.07321371, -0.11101919, ..., -0.14396206,
         -0.0499901 , -0.09893271],
        ...,
        [ 0.14955643, -0.00583661, -0.00119951, ..., -0.07104123,
         -0.0286596 , -0.03342304],
        [ 0.06839566, -0.12877463,  0.05618842, ..., -0.01180017,
          0.09284012, -0.12714407],
        [-0.03903491, -0.09870157,  0.07117632, ..., -0.02694204,
         -0.08345815, -0.12888123]], dtype=float32)>,
 <tf.Variable 'Dense_1/bias:0' shape=(256,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [6]:
#设置训练参数
total_steps = 0
max_episodes = 1000

# 记录训练数据 由于记录的不断增加，最终会将内存耗尽，所有在过程中要不断的保存结果
episode_time, navs, market_navs, diffs, episode_eps = [], [], [], [], []

In [7]:
def track_results(episode, nav_ma_100, nav_ma_10,
                  market_nav_100, market_nav_10,
                  win_ratio, total, epsilon):
    time_ma = np.mean([episode_time[-100:]])
    T = np.sum(episode_time)
    
    template = '{:>4d} | {} | Agent: {:>6.1%} ({:>6.1%}) | '
    template += 'Market: {:>6.1%} ({:>6.1%}) | '
    template += 'Wins: {:>5.1%} | eps: {:>6.3f}'
    print(template.format(episode, format_time(total), 
                          nav_ma_100-1, nav_ma_10-1, 
                          market_nav_100-1, market_nav_10-1, 
                          win_ratio, epsilon))

In [19]:
max_episodes # 训练过程为1000个episode

1000

In [20]:
max_episode_steps # 每个episode 走252步

252

In [20]:
# 开始训练
start = time()
results = [] #记录训练结果
episode_time, navs, market_navs, diffs, episode_eps = [], [], [], [], []

for episode in range(1, max_episodes + 1):
    # 每个episode开始前会将环境重置
    this_state = trading_environment.reset()
    

    # 每次从step1走到step252
    for episode_step in range(max_episode_steps):
        
        action = ddqn.epsilon_greedy_policy(this_state.reshape(-1, state_dim))
        next_state, reward, done, _ = trading_environment.step(action)
        
        # print("epi:", episode, "|step:", episode_step, "|action:", action, "|next_state:",next_state, "|reward:", reward, "done:", done)

        ddqn.memorize_transition(this_state, 
                                 action, 
                                 reward, 
                                 next_state, 
                                 0.0 if done else 1.0)
        if ddqn.train:
            ddqn.experience_replay()
        if done:
            break
        this_state = next_state
    
    # 记录训练时间
    episode_time.append(time() - start)
    # 记录每次epsilon
    episode_eps.append(ddqn.epsilon)

    # 用于记录每一步的训练结果 
    result = trading_environment.env.simulator.result()
    # 最后一步的训练结果
    final = result.iloc[-1]

    # 记录策略的NAV
    nav = final.nav * (1 + final.strategy_return)
    navs.append(nav)

     # 记录市场的NAV
    market_nav = final.market_nav
    market_navs.append(market_nav)
    
    # 计算策略与市场的差异
    diff = nav - market_nav
    diffs.append(diff)

    # 每隔10次打印训练结果
    if episode % 1 == 0:
        track_results(episode, np.mean(navs[-20:]), np.mean(navs[-10:]), 
                      np.mean(market_navs[-20:]), np.mean(market_navs[-10:]), 
                      np.sum([s > 0 for s in diffs[-20:]])/min(len(diffs), 20), 
                      episode_time[-1], episode_eps[-1])

    # 每隔50个回合存储训练的结果
    gap = 5
    if episode % gap == 0:
        print("保存训练的模型")
        # 保存训练的模型
        ddqn.save_model()

        # 保存统计数据
        index = np.floor(episode / gap)

        print("保存训练数据")

        print(list(range(int((index-1)*gap), episode+1)))
        print(episode_time)
        print(navs)
        print(market_navs)
        print(diffs)
        print(episode_eps)
        
        results = pd.DataFrame({'episode': list(range(int((index-1)*gap)+1, episode+1)),
                                'episode_time':episode_time,
                                'agent': navs,
                                'market': market_navs,
                                'difference': diffs,
                                'episode_eps':episode_eps
                                 }).set_index('episode')
        results['Strategy Wins (%)'] = (results.Difference > 0).rolling(gap).sum()
        
        if index == 0:
            results.to_csv(results_path / 'dnq_results.csv', index=False)
        else:
            results.to_csv(results_path / 'dnq_results.csv', mode='a', head=False, index=False) 
        episode_time, navs, market_navs, diffs, episode_eps = [], [], [], [], []

    # 当策略优于市场时训练结束
    if len(diffs) > 25 and all([r > 0 for r in diffs[-25:]]):
        print(result.tail())
        break

trading_environment.close()

1 | 00:00:23 | Agent: -18.8% (-18.8%) | Market:  26.6% ( 26.6%) | Wins:  0.0% | eps:  0.909
   2 | 00:00:42 | Agent: -28.0% (-28.0%) | Market:  25.2% ( 25.2%) | Wins:  0.0% | eps:  0.905
   3 | 00:01:01 | Agent: -26.7% (-26.7%) | Market:  38.7% ( 38.7%) | Wins:  0.0% | eps:  0.901
   4 | 00:01:19 | Agent: -18.4% (-18.4%) | Market:  36.7% ( 36.7%) | Wins:  0.0% | eps:  0.897
   5 | 00:01:38 | Agent: -23.4% (-23.4%) | Market:  41.1% ( 41.1%) | Wins:  0.0% | eps:  0.893
保存训练的模型
INFO:tensorflow:Assets written to: models/online_model/assets
INFO:tensorflow:Assets written to: models/online_model/assets
INFO:tensorflow:Assets written to: models/target_model/assets
INFO:tensorflow:Assets written to: models/target_model/assets
models have been saved.
保存训练数据
[0, 1, 2, 3, 4, 5]
[22.82962918281555, 41.841253995895386, 60.53480076789856, 79.12658619880676, 97.68651103973389]
[0.81188326710068, 0.6288693808617752, 0.758669811822195, 1.0663153926570423, 0.5643994510107292]
[1.26593736944849, 1.239005

ValueError: arrays must all be same length